# Pre-requisites

In [ ]:
pip install sentence_transformers

In [ ]:
import pandas as pd
import nltk
import re
import numpy as np
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
import gensim
import warnings
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk import everygrams
from nltk.stem import *
import itertools
from scipy import spatial
import collections
import json
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import CrossEncoder
from gensim.models import Word2Vec
import datetime
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

In [ ]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
%cd /content/gdrive/MyDrive/Kethan/CVE_CWE_Mapping/Cross_Encoder/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/gdrive/
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-fd5044aa27d0>", line 3, in <cell line: 3>
    get_ipython().run_line_magic('cd', '/content/gdrive/MyDrive/Kethan/CVE_CWE_Mapping/Cross_Encoder/')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  

In [ ]:
with open('./Data/230710_1/CWE_Desc_CrossEncoderL3SolutionV2.json', 'r') as cweFile:
    dict_mitre_view1003 = json.load(cweFile)

In [ ]:
df_train = pd.read_csv('./Data/230710_1/L3/train.csv')
df_val = pd.read_csv('./Data/230710_1/L3/val.csv')

# Model Training

In [ ]:
df_train.isna().sum(),df_train.shape

(cve.descriptions_list              6795
 cve.weaknesses_list                   0
 cwe.descriptions_list                 0
 is_similar                            0
 cve.descriptions_list_processed       0
 dtype: int64,
 (162935, 5))

In [ ]:
df_train.shape,df_val.shape

((162935, 5), (20367, 5))

In [ ]:
df_train.head(1)

,cve.descriptions_list,cve.weaknesses_list,cwe.descriptions_list,is_similar,cve.descriptions_list_processed
0,"It was discovered that read_file() in apport/hookutils.py would follow symbolic links or open FIFOs. When this function is used by the openjdk-17 package apport hooks, it could expose private data to other local users.",59,improp link resolut file access link follow insecur temporari file zip slip product attempt access file base filenam doe properli prevent filenam identifi link shortcut resolv unintend resourc,1,wa discov read file apport hookutil py would follow symbol link open fifo function use openjdk packag apport hook could expos privat data local user


In [ ]:
train_samples=[]
for index,row in df_train.iterrows():
    train_samples.append(InputExample(texts=[row['cve.descriptions_list_processed'], row['cwe.descriptions_list']], label=row['is_similar']))
    train_samples.append(InputExample(texts=[row['cwe.descriptions_list'], row['cve.descriptions_list_processed']], label=row['is_similar']))

dev_samples=[]
for index,row in df_val.iterrows():
    dev_samples.append(InputExample(texts=[row['cve.descriptions_list_processed'], row['cwe.descriptions_list']], label=row['is_similar']))
    dev_samples.append(InputExample(texts=[row['cwe.descriptions_list'], row['cve.descriptions_list_processed']], label=row['is_similar']))

In [ ]:
#Define our Cross-Encoder
train_batch_size = 32
num_epochs = 5
model_save_path = './ModelObjects/Temp/training_benchmark-'+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_save_path

'./ModelObjects/Temp/training_benchmark-2023-07-11_12-34-35'

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

In [ ]:
#evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='sts-dev')

In [ ]:
model = CrossEncoder('bert-base-uncased', num_labels=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
model.fit(train_dataloader=train_dataloader,
          #evaluator=evaluator,
          epochs=num_epochs,
          output_path=model_save_path)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10184 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10184 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10184 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10184 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10184 [00:00<?, ?it/s]

In [ ]:
#evaluator(model)
#model.save(model_save_path)

In [ ]:
#df_val = df_corpus[df_corpus['cve.id'].str.contains('CVE-2022')]

In [ ]:
df_val['prediction_similarity_score'] = list(model.predict(df_val[['cve.descriptions_list_processed','cwe.descriptions_list']].values.tolist()))

In [ ]:
df_val['prediction_similarity'] = df_val['prediction_similarity_score'].apply(lambda score: 1 if score>0.5 else 0)

In [ ]:
accuracy = np.mean(df_val['prediction_similarity']==df_val['is_similar'])
accuracy

0.9791329110816517

In [ ]:
#df_val.tail(2)

In [ ]:
df_val['prediction_similarity'].value_counts()

0    10219
1    10148
Name: prediction_similarity, dtype: int64

In [ ]:
df_val['is_similar'].value_counts()

0    10204
1    10163
Name: is_similar, dtype: int64

In [ ]:
df_val[df_val['prediction_similarity']!=df_val['is_similar']].shape

(425, 7)

In [ ]:
model.save(model_save_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-1310f296578c>", line 1, in <cell line: 1>
    model.save(model_save_path)
  File "/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py", line 315, in save
    self.model.save_pretrained(path)
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 1732, in save_pretrained
    os.makedirs(save_directory, exist_ok=True)
  File "/usr/lib/python3.10/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.10/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.10/os.py", line 225, in makedirs
    mkdir(name, mode)
OSError: [Errno 107] Transport endpoint is not connected: './ModelObjects'

During handling of the above exception, an

In [ ]:
df_val.columns

Index(['cve.descriptions_list', 'cve.weaknesses_list', 'cwe.descriptions_list',
       'is_similar', 'cve.descriptions_list_processed',
       'prediction_similarity_score', 'prediction_similarity'],
      dtype='object')

In [ ]:
df_val.head(1)

,cve.descriptions_list,cve.weaknesses_list,cwe.descriptions_list,is_similar,cve.descriptions_list_processed,prediction_similarity_score,prediction_similarity
64706,"Bonita Web 2021.2 is affected by a authentication/authorization bypass vulnerability due to an overly broad exclude pattern used in the RestAPIAuthorizationFilter. By appending ;i18ntranslation or /../i18ntranslation/ to the end of a URL, users with no privileges can access privileged API endpoints. This can lead to remote code execution by abusing the privileged API actions.",863,incorrect behavior order author pars canonic author bypass user control sql primari key author bypass user control key use non canon url path author decis guessabl captcha incorrect author intern asset expos unsaf debug access level state horizont author authz product perform author check actor attempt access resourc perform action doe correctli perform check allow attack bypass intend access restrict,1,bonita web affect authent author bypass vulner due overli broad exclud pattern use restapiauthorizationfilt append intransl intransl end url user privileg access privileg api endpoint lead remot code execut abus privileg api action,0.986959,1


In [ ]:
df_val[df_val['prediction_similarity']!=df_val['is_similar']][['cve.weaknesses_list','is_similar','prediction_similarity_score']]

,cve.weaknesses_list,is_similar,prediction_similarity_score
891,400,0,0.965081
895,400,0,0.992750
897,400,0,0.992797
3672,200,0,0.557984
3884,200,0,0.967949
...,...,...,...
30541,345,0,0.567938
30988,610,1,0.244335
31131,662,1,0.068080
31132,662,1,0.057853


In [ ]:
df_val.to_excel("")

In [ ]:
#!zip -r ./Cross_Encoder.zip /content/D:/AI_ML/kethan/CVE_CWE_Mapping/Output/CrossEncoder/training_benchmark-2023-06-05_12-22-10/

# Inference

In [ ]:
model = CrossEncoder('./ModelObjects/Temp/training_benchmark-2023-06-20_06-47-15/')

In [ ]:
df_test = df_corpus[(df_corpus['cve.id'].str.contains('CVE-2023')|df_corpus['cve.id'].str.contains('CVE-2022'))
 & (df_corpus['is_similar']==1)]

In [ ]:
df_test.shape

(3445, 6)

In [ ]:
counter=0
def return_predictions(row):
  global counter
  counter+=1
  print('\r Counter value: {0} of {1}'.format(str(counter),str(df_test.shape[0])),end='')
  #print("Counter value: "+str(counter)+" of "+str(df_test.shape[0]))
  similarities ={}
  for key,val in dict_mitre_view1003.items():
      similarities[key] = model.predict([val,row['cve.descriptions_list_processed']])
  sorted_similarities = sorted(similarities.items(), key=lambda x:x[1], reverse=True)
  sorted_similarities = [val[0] for val in sorted_similarities]
  sorted_similarities = sorted_similarities[:10]
  return sorted_similarities

In [ ]:
df_test['predicted_values_lst'] = df_test.apply(lambda row: return_predictions(row),axis=1)

 Counter value: 3445 of 3445

In [ ]:
df_test['predicted_values_rank'] = df_test.apply(lambda row: row['predicted_values_lst'].index(str(row['cve.weaknesses_list'])) if str(row['cve.weaknesses_list']) in row['predicted_values_lst']  else 10,axis=1)

In [ ]:
df_test.head(5)

,cve.id,cve.descriptions_list,cve.weaknesses_list,cwe.descriptions_list,is_similar,cve.descriptions_list_processed,predicted_values_lst,predicted_values_rank
509,CVE-2022-21670,"markdown-it is a Markdown parser. Prior to version 1.3.2, special patterns with length greater than 50 thousand characterss could slow down the parser significantly. Users should upgrade to version 12.3.2 to receive a patch. There are no known workarounds aside from upgrading.",400,uncontrol resourc consumpt alloc resourc without limit throttl miss refer activ alloc resourc miss refer activ file descriptor handl alloc file descriptor handl without limit throttl log excess data memori alloc excess size valu improp restrict power consumpt incorrect use autobox unbox perform critic oper improp write handl limit write non volatil memori improperli control sequenti memori alloc resourc exhaust file descriptor exhaust stack exhaust stack exhaust product doe properli control alloc mainten limit resourc therebi enabl actor influenc amount resourc consum eventu lead exhaust avail resourc,1,markdown markdown parser prior version special pattern length greater thousand charact could slow parser significantli user upgrad version receiv patch known workaround asid upgrad,"[20, 682, 863, 436, 407, 269, 326, 704, 345, 669]",10
510,CVE-2022-21839,Windows Event Tracing Discretionary Access Control List Denial of Service Vulnerability.,400,uncontrol resourc consumpt alloc resourc without limit throttl miss refer activ alloc resourc miss refer activ file descriptor handl alloc file descriptor handl without limit throttl log excess data memori alloc excess size valu improp restrict power consumpt incorrect use autobox unbox perform critic oper improp write handl limit write non volatil memori improperli control sequenti memori alloc resourc exhaust file descriptor exhaust stack exhaust stack exhaust product doe properli control alloc mainten limit resourc therebi enabl actor influenc amount resourc consum eventu lead exhaust avail resourc,1,window event trace discretionari access control list denial servic vulner,"[20, 345, 362, 732, 330, 436, 287, 668, 706, 610]",10
511,CVE-2022-21843,"Windows IKE Extension Denial of Service Vulnerability. This CVE ID is unique from CVE-2022-21848, CVE-2022-21883, CVE-2022-21889, CVE-2022-21890.",400,uncontrol resourc consumpt alloc resourc without limit throttl miss refer activ alloc resourc miss refer activ file descriptor handl alloc file descriptor handl without limit throttl log excess data memori alloc excess size valu improp restrict power consumpt incorrect use autobox unbox perform critic oper improp write handl limit write non volatil memori improperli control sequenti memori alloc resourc exhaust file descriptor exhaust stack exhaust stack exhaust product doe properli control alloc mainten limit resourc therebi enabl actor influenc amount resourc consum eventu lead exhaust avail resourc,1,window ike extens denial servic vulner cve id uniqu cve cve cve cve,"[20, 345, 119, 436, 672, 754, 269, 669, 704, 682]",10
512,CVE-2022-21847,Windows Hyper-V Denial of Service Vulnerability.,400,uncontrol resourc consumpt alloc resourc without limit throttl miss refer activ alloc resourc miss refer activ file descriptor handl alloc file descriptor handl without limit throttl log excess data memori alloc excess size valu improp restrict power consumpt incorrect use autobox unbox perform critic oper improp write handl limit write non volatil memori improperli control sequenti memori alloc resourc exhaust file descriptor exhaust stack exhaust stack exhaust product doe properli control alloc mainten limit resourc therebi enabl actor influenc amount resourc consum eventu lead exhaust avail resourc,1,window hyper v denial servic vulner,"[20, 345, 400, 119, 436, 672, 754, 404, 682, 665]",2
513,CVE-2022-21848,"Windows IKE Extension Denial of Service Vulnerability. This CVE ID is unique from CVE-2022-21843, CVE-2022-21883, CVE-2022-21889, CVE-2022-218

In [ ]:
limit=10
scores = pd.DataFrame()
scores['Rank']= range(0,limit)

def return_accuracy(counter,limit,levelName='Unnamed'):
    global scores
    for top_rank_limit in range(limit):
        correct = np.sum([i[1] for i in counter.items()  if i[0]<=top_rank_limit])
        total = np.sum([i[1] for i in counter.items()])
        accur = correct/total
        scores.loc[top_rank_limit,levelName] = accur

return_accuracy(collections.Counter(df_test['predicted_values_rank']),limit)
scores

,Rank,Unnamed
0,0,0.341074
1,1,0.468795
2,2,0.545718
3,3,0.592743
4,4,0.629318
5,5,0.664731
6,6,0.689695
7,7,0.719884
8,8,0.741945
9,9,0.764296


In [ ]:
df_test.to_csv('./Output/Temp/220620_22_23_Predictions.csv',index=False)